In [1]:
import pandas as pd
import geopandas as gpd
import os
from osgeo import ogr
import numpy as np

#### Import Transport Data and filter

##### Global steps for filtering data:
- Change the columns name to avoid space
- Only keep relevant column since unit are stored in overview of data
- Filters the data for a chosen year (e.g. 2023)
- Keeps only EU27 countries
- Calculate share of countries with the global value EU27_2020 

In [3]:
file_path = r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND\Data\estat_avia_paexcc.tsv.gz"
chosen_time = '2024-Q1'
data = pd.read_csv(file_path, compression='gzip', delimiter='\t')
data.columns = data.columns.str.strip()

In [4]:
data['geo'] = data[data.columns[0]].str.split(',').str[-1]
data = data.drop(columns=[data.columns[0]]) # drop everything except geo

In [5]:
def detect_country_column(df):
    for col in df.columns:
        if 'geo\\TIME_PERIOD' in col or 'geo' in col or 'country_code' in col:
            return col
    raise ValueError("Could not find the correct country column.")

In [6]:
detect_country_column(data)

'geo'

In [7]:
EU27_COUNTRIES = [
    'AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'EL','HR',
    'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'EU27_2020'
]

In [8]:
def filter_eu27_year(df, time):
    country_column = detect_country_column(df)
    df['country_code'] = df[country_column].str.extract(r'([A-Z0-9_]+)$')  # Supports EU27_2020
    value_columns = df.columns.difference([country_column, 'country_code']).tolist()

    # Keep only countries that are in the eu27
    df = df[df['country_code'].isin(EU27_COUNTRIES)]

    # Find missing eu27 countries
    missing_countries = set(EU27_COUNTRIES) - set(df['country_code'])

    missing_rows = pd.DataFrame({
        country_column: [df[country_column].iloc[0].rsplit(',', 1)[0] + ',' + country for country in missing_countries],
        'country_code': list(missing_countries)
    })

    for col in value_columns:
        missing_rows[col] = np.nan  # Fill missing values with NaN

    df = pd.concat([df, missing_rows], ignore_index=True)
    df = df[df['country_code'].isin(EU27_COUNTRIES)]

    # Keep only the year we analyze
    year_columns = [col for col in df.columns if str(col).isdigit()]
    df = df[['country_code'] + [str(time)]]

    # Clean the values to keep only numeric data (remove letters or any non-numeric characters)
    df[str(time)] = df[str(time)].apply(lambda x: ''.join(filter(str.isdigit, str(x))) if isinstance(x, str) else x)
    df[str(time)] = pd.to_numeric(df[str(time)], errors='coerce')  # Convert to numeric, NaN if there's an error

    eu27_row = df[df['country_code'] == 'EU27_2020']
    
    if eu27_row[str(time)].isna().all():  # If EU27_2020 value is NaN
        eu27_value = df[df['country_code'] != 'EU27_2020'][str(time)].sum()  # Sum all country values
        df.loc[df['country_code'] == 'EU27_2020', str(time)] = eu27_value  # Assign sum to EU27_2020
    else:
        eu27_value = eu27_row[str(time)].values[0]

    # Compute the share
    df[f'share_of_{str(time)}'] = df[str(time)] / eu27_value

    return df

In [9]:
data_eu27 = filter_eu27_year(data, chosen_time)
data_eu27

,country_code,2024-Q1,share_of_2024-Q1
0,AT,2753330.0,0.027502
1,BE,2353086.0,0.023504
2,BG,646618.0,0.006459
3,CY,658072.0,0.006573
4,CZ,1280851.0,0.012794
5,DE,17069530.0,0.170498
6,DK,2503432.0,0.025005
7,EE,149926.0,0.001498
8,EL,1562752.0,0.015609
9,ES,16106237.0,0.160877


#### Merging and Summing Transport Data

The sections of code below combine two datasets to get total values for a transport category — for example, heavy goods vehicles by weight.

##### Steps for summing two categories:
- Merges the two datasets by country and unit
- Sums their values to get totals
- Filters the data for a chosen year (e.g. 2023)
- Keeps only EU27 countries
- Calculates each country’s share of the total EU27 value (available)

In [ ]:
file_path2 = r"C:\Users\mar.eco\Desktop\REMIND\Data\estat_avia_paincc.tsv.gz"
data2 = pd.read_csv(file_path2, compression='gzip', delimiter='\t')
data2.columns = data2.columns.str.strip()

In [ ]:
first_col = data2.columns[0]
data2['geo'] = data2[first_col].str.split(',').str[-1]
data2 = data2.drop(columns=[first_col])
data2

In [ ]:
data2_eu27 = filter_eu27_year(data2, chosen_time)
data2_eu27

In [ ]:
def merge_and_sum_datasets(df1, df2):
    country_column1 = detect_country_column(df1)  
    value_columns1 = df1.columns.difference([country_column1]).tolist()

    country_column2 = detect_country_column(df2)  
    value_columns2 = df2.columns.difference([country_column2]).tolist()

    print(f"Column in df1: {country_column1}, Column in df2: {country_column2}")
    
    # Create an empty DataFrame with the same columns as df1
    merged_df = pd.DataFrame(columns=df1.columns)
    all_countries = set(df1[country_column1])
    
    for country in all_countries:
        row1 = df1[df1[country_column1] == country]
        row2 = df2[df2[country_column2] == country]
        new_row = {country_column1: country}
        
        for col in value_columns1:
            value1 = pd.to_numeric(row1[col], errors='coerce').sum()
            value2 = pd.to_numeric(row2[col], errors='coerce').sum()
            new_row[col] = value1 + value2  # Sum the values
        
        # Append the new row to merged_df
        merged_df = pd.concat([merged_df, pd.DataFrame([new_row])], ignore_index=True)

    return merged_df

In [ ]:
data_merged = merge_and_sum_datasets(data, data2)
data_merged

In [ ]:
data_final = filter_eu27_year(data_merged, chosen_time)
data_final

#### Filling in Missing Country Data (MT)

This part of the code handles a situation where:
- The EU27 total value is available
- One country's data is missing (here Malta - `MT`) 

##### Steps detailed:
1. Get the EU27 total for the selected year
2. Sum up the values of all other countries
3. Calculates missing country value by subtracting the others from the EU27 total
4. Updates Malta's row in the data with:
   - The estimated value
   - Its share of the EU27 total

In [ ]:
eu27_value = data_final.loc[data_final['country_code'] == 'EU27_2020', str(chosen_time)].values[0]
sum_other_countries = data_final.loc[(data_final['country_code'] != 'EU27_2020') & 
                                        (data_final['country_code'] != 'MT'), str(chosen_time)].sum()
mt_value = eu27_value - sum_other_countries
if mt_value < 1 :
    mt_value = 0
# Update the existing MT row instead of appending
data_final.loc[data_final['country_code'] == 'MT', str(chosen_time)] = mt_value
data_final.loc[data_final['country_code'] == 'MT', f'share_of_{str(chosen_time)}'] = mt_value / eu27_value
data_final

#### Saving final dataset for further use

##### Steps detailed:
1. Get original path given
2. Create folder for results
3. Rename the final file path
4. Save the dataset 

In [ ]:
def save_final_dataframe(df, file_path):
    directory, filename = os.path.split(file_path)   # original path 

    final_folder = os.path.join(directory, "Final")
    os.makedirs(final_folder, exist_ok=True)  # This will create the folder if it doesn't already exist

    new_file_path = os.path.join(final_folder, filename)
    
    df.to_csv(new_file_path, sep='\t', compression='gzip', index=False)
    
    print(f"File saved successfully to: {new_file_path}")
    return new_file_path

In [ ]:
path_final = r"C:\Users\mar.eco\Desktop\REMIND\Data\estat_avia_international.tsv.gz"
data_final = data_final
df_result = filter_eu27_year(data_final, chosen_time)
new_file_path = save_final_dataframe(df_result, path_final)